In [92]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import category_encoders as ce
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import LabelEncoder
from prophet import Prophet
warnings.filterwarnings("ignore")

LOAD DỮ LIỆU

In [93]:
df = pd.read_csv("train.csv")
print(f"Data shape: {df.shape}")
print(f"Columns: {len(df.columns)}")

Data shape: (1460, 81)
Columns: 81


ĐỊNH NGHĨA TIỆN ÍCH

In [94]:
# Train/Test split theo năm
df_train = df[df['YrSold'] < 2010].copy()
df_test  = df[df['YrSold'] >= 2010].copy()

# Xử lý giá trị thiếu
def basic_impute(df_in):
    df = df_in.copy()

    for c in df.select_dtypes(include=np.number):
        df[c] = df[c].fillna(df[c].median())

    for c in df.select_dtypes(include=["object", "category"]):
        df[c] = df[c].fillna(df[c].mode().iloc[0] if not df[c].mode().empty else "NA")

    return df

def print_strategy_summary(name, train_df, test_df, n_preview=5):
    print(f"{name.upper()} RESULTS")
    print(f"Train shape : {train_df.shape}")
    print(f"Test shape  : {test_df.shape}")
    print(f"Features    : {train_df.shape[1] - 1}")
    assert list(train_df.columns) == list(test_df.columns), "Column mismatch!"

quality_map = {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, np.nan: 0}

location_lot_cols = [
    "MSSubClass", "MSZoning", "LotFrontage", "LotArea",
    "Street", "Alley", "LotShape", "LandContour",
    "Utilities", "LotConfig", "LandSlope",
    "Neighborhood", "Condition1", "Condition2"
]

size_interior_cols = [
    "GrLivArea", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF",
    "LowQualFinSF", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF",
    "TotRmsAbvGrd", "BedroomAbvGr", "KitchenAbvGr",
    "GarageArea", "GarageCars", "MasVnrArea"
]

quality_condition_cols = [
    "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd",
    "ExterQual", "ExterCond", "BsmtQual", "BsmtCond",
    "HeatingQC", "KitchenQual", "FireplaceQu",
    "GarageQual", "GarageCond", "PoolQC"
]

amenities_sale_time_cols = [
    "Fireplaces", "GarageYrBlt", "PoolArea", "Fence",
    "MiscVal", "MoSold", "YrSold", "SaleType", "SaleCondition"
]

groups_info = {
    "Location & Lot": location_lot_cols,
    "Size & Interior": size_interior_cols,
    "Quality & Condition": quality_condition_cols,
    "Amenities, Sale & Time": amenities_sale_time_cols
}

print(f"Train: {len(df_train)} rows ({len(df_train)/len(df)*100:.1f}%)")
print(f"Test:  {len(df_test)} rows ({len(df_test)/len(df)*100:.1f}%)")
print(f"\nTrain years: {df_train['YrSold'].min()} - {df_train['YrSold'].max()}")
print(f"Test years:  {df_test['YrSold'].min()} - {df_test['YrSold'].max()}")
print("\nFeature groups loaded:")
print(f"Location & Lot:     {len(location_lot_cols)}")
print(f"Size & Interior:    {len(size_interior_cols)}")
print(f"Quality & Condition:{len(quality_condition_cols)}")
print(f"Amenities/Sale:     {len(amenities_sale_time_cols)}")

Train: 1285 rows (88.0%)
Test:  175 rows (12.0%)

Train years: 2006 - 2009
Test years:  2010 - 2010

Feature groups loaded:
Location & Lot:     14
Size & Interior:    14
Quality & Condition:14
Amenities/Sale:     9


TIỀN XỬ LÝ DỮ LIỆU

In [ ]:
# 1. One-Hot + Standard
def strategy1_onehot_standard(df_in, reference_columns=None):
    df = basic_impute(df_in)

    cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
    num_cols = [c for c in df.columns if c not in cat_cols and c != "SalePrice"]

    df_cat = pd.get_dummies(df[cat_cols].astype(str))

    scaler = StandardScaler()
    df_num = pd.DataFrame(
        scaler.fit_transform(df[num_cols]),
        columns=num_cols,
        index=df.index
    )

    result = pd.concat([df_num, df_cat], axis=1)

    if reference_columns is not None:
        result = result.reindex(columns=reference_columns.drop("SalePrice"), fill_value=0)

    result["SalePrice"] = df["SalePrice"]
    return result

STRATEGY 1 – ONEHOT + STANDARD RESULTS
Train shape : (1285, 287)
Test shape  : (175, 287)
Features    : 286


In [ ]:
# 2. Target + Robust
def strategy2_target_robust(df_in, reference_columns=None):
    df = basic_impute(df_in)
    target = "SalePrice"

    cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
    num_cols = [c for c in df.columns if c not in cat_cols and c != target]

    te = ce.TargetEncoder(cols=cat_cols, smoothing=0.3)
    df_cat = te.fit_transform(df[cat_cols], df[target])

    scaler = RobustScaler()
    df_num = pd.DataFrame(
        scaler.fit_transform(df[num_cols]),
        columns=num_cols,
        index=df.index
    )

    result = pd.concat([df_num, df_cat], axis=1)

    if reference_columns is not None:
        result = result.reindex(columns=reference_columns.drop(target), fill_value=0)

    result[target] = df[target]
    return result

STRATEGY 2 – TARGET + ROBUST RESULTS
Train shape : (1285, 81)
Test shape  : (175, 81)
Features    : 80


In [ ]:
# 3. PCA + Ordinal
def strategy3_pca_ordinal(df_in, reference_columns=None):
    df = basic_impute(df_in)

    pca_components = 3  # keyword-only internally

    size_cols = [c for c in size_interior_cols if c in df.columns]

    if size_cols:
        scaler = StandardScaler()
        pca = PCA(n_components=min(pca_components, len(size_cols)))
        size_pca = pca.fit_transform(scaler.fit_transform(df[size_cols]))

        size_df = pd.DataFrame(
            size_pca,
            columns=[f"SizePCA{i+1}" for i in range(size_pca.shape[1])],
            index=df.index
        )
    else:
        size_df = pd.DataFrame(index=df.index)

    qual_df = pd.DataFrame(index=df.index)
    for c in quality_condition_cols:
        if c in df.columns:
            if df[c].dtype == object:
                qual_df[c + "_ord"] = df[c].map(quality_map).fillna(0).astype(int)
            else:
                qual_df[c] = df[c]

    other_cols = [
        c for c in df.select_dtypes(include=[np.number]).columns
        if c not in size_cols + quality_condition_cols + ["SalePrice"]
    ]

    other_df = (
        pd.DataFrame(
            StandardScaler().fit_transform(df[other_cols]),
            columns=other_cols,
            index=df.index
        ) if other_cols else pd.DataFrame(index=df.index)
    )

    result = pd.concat([size_df, qual_df, other_df], axis=1)

    if reference_columns is not None:
        result = result.reindex(columns=reference_columns.drop("SalePrice"), fill_value=0)

    result["SalePrice"] = df["SalePrice"]
    return result

STRATEGY 3 – PCA + ORDINAL RESULTS
Train shape : (1285, 37)
Test shape  : (175, 37)
Features    : 36


In [ ]:

# 4. MinMax + Embedding
def strategy4_minmax_embedding(df_in, reference_columns=None):
    df = basic_impute(df_in)

    num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c != "SalePrice"]
    num_df = pd.DataFrame(
        MinMaxScaler().fit_transform(df[num_cols]),
        columns=num_cols,
        index=df.index
    )

    cat_df = pd.DataFrame(index=df.index)

    for c in df.select_dtypes(include=["object", "category"]).columns:
        if df[c].nunique() > 10:
            cat_df[c + "_idx"] = pd.factorize(df[c])[0]
        else:
            cat_df = pd.concat([cat_df, pd.get_dummies(df[c], prefix=c)], axis=1)

    result = pd.concat([num_df, cat_df], axis=1)

    if reference_columns is not None:
        result = result.reindex(columns=reference_columns.drop("SalePrice"), fill_value=0)

    result["SalePrice"] = df["SalePrice"]
    return result

STRATEGY 4 – MINMAX + EMBEDDING RESULTS
Train shape : (1285, 235)
Test shape  : (175, 235)
Features    : 234


In [118]:
STRATEGIES = {
    "Strategy 1 – OneHot + Standard": strategy1_onehot_standard,
    "Strategy 2 – Target + Robust": strategy2_target_robust,
    "Strategy 3 – PCA + Ordinal": strategy3_pca_ordinal,
    "Strategy 4 – MinMax + Embedding": strategy4_minmax_embedding,
}

processed_data = {}

for name, func in STRATEGIES.items():
    train = func(df_train)
    test  = func(df_test, reference_columns=train.columns)
    processed_data[name] = (train, test)
    print()

    print_strategy_summary(name, train, test)



STRATEGY 1 – ONEHOT + STANDARD RESULTS
Train shape : (1285, 287)
Test shape  : (175, 287)
Features    : 286

STRATEGY 2 – TARGET + ROBUST RESULTS
Train shape : (1285, 81)
Test shape  : (175, 81)
Features    : 80

STRATEGY 3 – PCA + ORDINAL RESULTS
Train shape : (1285, 37)
Test shape  : (175, 37)
Features    : 36

STRATEGY 4 – MINMAX + EMBEDDING RESULTS
Train shape : (1285, 235)
Test shape  : (175, 235)
Features    : 234


In [121]:
def evaluate_strategy(name, train_df, test_df, alpha=1.0):
    X_train = train_df.drop(columns="SalePrice")
    y_train = train_df["SalePrice"]
    X_test  = test_df.drop(columns="SalePrice")
    y_test  = test_df["SalePrice"]

    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    return {
        "Strategy": name,
        "MAE": mean_absolute_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "R2": r2_score(y_test, y_pred),
        "Features": X_train.shape[1]
    }

In [122]:
results = []

for name, (train_df, test_df) in processed_data.items():
    res = evaluate_strategy(name, train_df, test_df)
    results.append(res)

results_strategy_df = pd.DataFrame(results)
results_strategy_df


,Strategy,MAE,RMSE,R2,Features
0,Strategy 1 – OneHot + Standard,4.877329e+04,5.333553e+04,5.579656e-01,286
1,Strategy 2 – Target + Robust,1.045836e+08,1.045942e+08,-1.699956e+06,80
2,Strategy 3 – PCA + Ordinal,1.594893e+05,1.624179e+05,-3.099122e+00,36
3,Strategy 4 – MinMax + Embedding,1.105623e+05,1.171450e+05,-1.132408e+00,234


In [123]:
def select_best_strategy(results_df):
    df = results_df.copy()

    # Normalize (lower is better)
    df["MAE_norm"]  = (df["MAE"]  - df["MAE"].min())  / (df["MAE"].max()  - df["MAE"].min())
    df["RMSE_norm"] = (df["RMSE"] - df["RMSE"].min()) / (df["RMSE"].max() - df["RMSE"].min())

    # Normalize R2 (higher is better → đảo chiều)
    df["R2_norm"] = (df["R2"].max() - df["R2"]) / (df["R2"].max() - df["R2"].min())

    # Final score
    df["FinalScore"] = df["MAE_norm"] + df["RMSE_norm"] + df["R2_norm"]

    best_row = df.loc[df["FinalScore"].idxmin()]

    print("BEST STRATEGY (MULTI-METRIC)")
    print(f"Strategy    : {best_row['Strategy']}")
    print(f"Final Score : {best_row['FinalScore']:.4f}")
    print(f"MAE         : {best_row['MAE']:,.2f}")
    print(f"RMSE        : {best_row['RMSE']:,.2f}")
    print(f"R²          : {best_row['R2']:.4f}")

    return best_row, df.sort_values("FinalScore")


In [128]:
best_strategy, ranked_df = select_best_strategy(results_strategy_df)

ranked_df[[
    "Strategy",
    "MAE",
    "RMSE",
    "R2",
    "Features",
    "FinalScore"
]]


BEST STRATEGY (MULTI-METRIC)
Strategy    : Strategy 1 – OneHot + Standard
Final Score : 0.0000
MAE         : 48,773.29
RMSE        : 53,335.53
R²          : 0.5580


,Strategy,MAE,RMSE,R2,Features,FinalScore
0,Strategy 1 – OneHot + Standard,4.877329e+04,5.333553e+04,5.579656e-01,286,0.000000
3,Strategy 4 – MinMax + Embedding,1.105623e+05,1.171450e+05,-1.132408e+00,234,0.001202
2,Strategy 3 – PCA + Ordinal,1.594893e+05,1.624179e+05,-3.099122e+00,36,0.002105
1,Strategy 2 – Target + Robust,1.045836e+08,1.045942e+08,-1.699956e+06,80,3.000000


In [164]:
best_strategy_name = best_strategy["Strategy"]
best_train_df, best_test_df = processed_data[best_strategy_name]

print("Using strategy:", best_strategy_name)


Using strategy: Strategy 1 – OneHot + Standard


In [165]:
# Chỉ lấy các cột numeric (không tính SalePrice)
numeric_cols = best_train_df.select_dtypes(include=np.number).columns.tolist()
numeric_cols.remove("SalePrice")
# Ma trận tương quan
corr_matrix = best_train_df[numeric_cols + ["SalePrice"]].corr()

# Lấy tương quan với SalePrice
corr_target = corr_matrix["SalePrice"].sort_values(ascending=False)
print("Correlation with SalePrice:\n", corr_target)


Correlation with SalePrice:
 SalePrice        1.000000
OverallQual      0.791936
GrLivArea        0.707788
GarageCars       0.641778
GarageArea       0.625583
TotalBsmtSF      0.604576
1stFlrSF         0.600848
FullBath         0.565798
TotRmsAbvGrd     0.528824
YearBuilt        0.516608
YearRemodAdd     0.510059
Fireplaces       0.470974
MasVnrArea       0.467104
GarageYrBlt      0.463615
BsmtFinSF1       0.367683
LotFrontage      0.344275
WoodDeckSF       0.335614
2ndFlrSF         0.326335
OpenPorchSF      0.318226
HalfBath         0.282453
LotArea          0.256522
BsmtFullBath     0.228524
BsmtUnfSF        0.219387
BedroomAbvGr     0.171937
ScreenPorch      0.115762
PoolArea         0.098235
MoSold           0.050079
3SsnPorch        0.045863
BsmtFinSF2       0.003293
BsmtHalfBath    -0.014173
MiscVal         -0.018843
LowQualFinSF    -0.024876
YrSold          -0.025487
Id              -0.033136
MSSubClass      -0.071855
OverallCond     -0.071976
EnclosedPorch   -0.128491
KitchenAb

In [167]:
selected_features = corr_target[abs(corr_target) >= 0.5].index.tolist()
selected_features.remove("SalePrice")  # bỏ cột target
print("Selected regressors for Prophet:", selected_features)

Selected regressors for Prophet: ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd']
